In [ ]:

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)


import os
import os
import math
import time
import random
import shutil
from pathlib import Path
from contextlib import contextmanager
from collections import defaultdict, Counter

import scipy as sp
import numpy as np
import pandas as pd

from tqdm.auto import tqdm
from functools import partial

import cv2
from PIL import Image

from matplotlib import pyplot as plt

import torch
import torch.nn as nn
import torch.nn.functional as F

OUTPUT_DIR = './'
if not os.path.exists(OUTPUT_DIR):
    os.makedirs(OUTPUT_DIR)

TEST_PATH = '../input/ranzcr-clip-catheter-line-classification/test'


In [ ]:
import datetime
import numpy as np 
import pandas as pd
from time import time
from numba import njit
from sklearn.metrics import log_loss,roc_auc_score
from scipy.optimize import minimize, LinearConstraint, dual_annealing

In [ ]:
folds = [0,1,2,3,4]
target_size=11
target_cols=['ETT - Abnormal', 'ETT - Borderline', 'ETT - Normal',
                 'NGT - Abnormal', 'NGT - Borderline', 'NGT - Incompletely Imaged', 'NGT - Normal', 
                 'CVC - Abnormal', 'CVC - Borderline', 'CVC - Normal',
                 'Swan Ganz Catheter Present']
cols = [f'preds_{c}' for c in target_cols]



In [ ]:
from contextlib import contextmanager

@contextmanager
def timer(name):
    t0 = time.time()
    LOGGER.info(f'[{name}] start')
    yield
    LOGGER.info(f'[{name}] done in {time.time() - t0:.0f} s.')


def init_logger(log_file=OUTPUT_DIR+'inference.log'):
    from logging import getLogger, INFO, FileHandler,  Formatter,  StreamHandler
    logger = getLogger(__name__)
    logger.setLevel(INFO)
    handler1 = StreamHandler()
    handler1.setFormatter(Formatter("%(message)s"))
    handler2 = FileHandler(filename=log_file)
    handler2.setFormatter(Formatter("%(message)s"))
    logger.addHandler(handler1)
   # logger.addHandler(handler2)
    return logger

LOGGER = init_logger()


def seed_torch(seed=42):
    random.seed(seed)
    os.environ['PYTHONHASHSEED'] = str(seed)
    np.random.seed(seed)
    torch.manual_seed(seed)
    torch.cuda.manual_seed(seed)
    torch.backends.cudnn.deterministic = True

seed_torch(seed=42)

In [ ]:
def rank_data(sub2):
    sub['target'] = sub2
    sub['target'] = sub['target'].rank() / sub['target'].rank().max()
    return sub2

In [ ]:
def obj_func(x, *oof_list):
    oof = 0
    x[x<0] = 0
    x = x/np.sum(x)
    for i in range(min(len(x), len(oof_list))):
        oof += x[i]*oof_list[i]['predict']
    ground_truth = oof_list[0]['target']
    return -roc_auc_score(ground_truth, oof)

In [ ]:
# ====================================================
# Utils
# ====================================================
def oof_score(oof_df):
    def get_score(y_true, y_pred):
        scores = []
        for i in range(y_true.shape[1]):
            score = roc_auc_score(y_true[:,i], y_pred[:,i])
            scores.append(score)
        avg_score = np.mean(scores)
        return avg_score
    def get_result(result_df):
        preds = result_df[[f'preds_{c}' for c in target_cols]].values
        labels = result_df[target_cols].values
        score = get_score(labels, preds)
        #LOGGER.info(f'Score: {score:<.4f}  Scores: {np.round(scores, decimals=4)}')
        return score
    for fold in folds:
        fold_oof_df = oof_df[oof_df['fold']==fold].reset_index(drop=True)
        #LOGGER.info(f"========== fold: {fold} result ==========")
       # get_result(fold_oof_df)
    return get_result(fold_oof_df)

In [ ]:
#oof1 = pd.read_csv('../input/multihead-selfattn-inception-eb5/3rdstage_self_attn_allfold_ps8_resnet200d_320/3rdstage_self_attn_allfold_ps8_resnet200d_320/oof_df.csv')



#oof2 = pd.read_csv('../input/multihead-selfattn-inception-eb5/ns3rdstagefoldallpsdo8_inception/ns3rdstagefoldallpsdo8_inception/oof_df.csv')

oof3 = pd.read_csv('../input/multihead-selfattn-inception-eb5/ns3rdstagefoldallpsdo8_ef7/ns3rdstagefoldallpsdo8_ef7/oof_df.csv')


oof1 = pd.read_csv('../input/resnet200d-967-oof/oof.csv')
oof2 = pd.read_csv('../input/seresnet-965pub-768-oof/oof.csv')
oof4 = pd.read_csv('../input/multiheadresnet200dpseudooof/oof.csv')
oof5 = pd.read_csv('../input/efn-mh-bestcv-963-oof/oof.csv')
oof6 = pd.read_csv('../input/resnet200d-mh-966-oof-np/oof.csv')

In [ ]:
oof2 = oof2.set_index('StudyInstanceUID')
oof2 = oof2.reindex(index=oof1['StudyInstanceUID'])
oof2 = oof2.reset_index()

In [ ]:
oof3 = oof3.set_index('StudyInstanceUID')
oof3 = oof3.reindex(index=oof1['StudyInstanceUID'])
oof3 = oof3.reset_index()

In [ ]:
oof4 = oof4.set_index('StudyInstanceUID')
oof4 = oof4.reindex(index=oof1['StudyInstanceUID'])
oof4 = oof4.reset_index()

In [ ]:
oof5

In [ ]:
oof5 = oof5.set_index('StudyInstanceUID')
oof5 = oof5.reindex(index=oof1['StudyInstanceUID'])
oof5 = oof5.reset_index()

In [ ]:
oof6 = oof6.set_index('StudyInstanceUID')
oof6 = oof6.reindex(index=oof1['StudyInstanceUID'])
oof6 = oof6.reset_index()

In [ ]:
#y_true = pd.read_csv('../input/lish-moa/train_targets_scored.csv', index_col = 'sig_id').values

In [ ]:
def oof_ranking(oof):
    for i in cols:
        oof[i] = oof[i].rank() / oof[i].rank().max()
        #oof[i] = oof[i].rank() / oof[i].rank().max()
    return oof

In [ ]:
oof1.shape

In [ ]:
oof2.shape

In [ ]:
oof2.columns = oof1.columns
oof3.columns = oof1.columns
oof4.columns = oof1.columns
oof5.columns = oof1.columns
oof6.columns = oof1.columns

In [ ]:
oof3['fold']

In [ ]:
# oof1 = oof_ranking(oof1)
# oof2 = oof_ranking(oof2)
# oof3 = oof_ranking(oof3)
# oof4 = oof_ranking(oof4)
# oof5 = oof_ranking(oof5)

In [ ]:
weights = [0.25,0.25,0.25,0.25,0.25]

In [ ]:

def func(weights):
    coef = 1e-6
    oof_blend = oof1.copy()
    cols = [f'preds_{c}' for c in target_cols]
    
    oof_blend[cols] = weights[0] * oof1[cols].values + weights[1] * oof2[cols].values + weights[2] * oof3[cols].values + weights[3] * oof4[cols].values+ weights[4] * oof5[cols].values
                      

#     score = log_loss_metric(oof_blend)
    score = oof_score(oof_blend)
    penalty = coef * (np.sum(weights) - 1) ** 2
    return score 

In [ ]:
func(weights)

In [ ]:
model_oofs = [oof1,oof2,oof3,oof4,oof5,oof6]

In [ ]:
def objective(trial):
        weights = []
        blends = oof1.copy()
        for i in range(len(model_oofs)):
            weights.append(trial.suggest_float(f"w{i}", 0, 1.0))

        blend = np.zeros(model_oofs[0][cols].shape)
        for i in range(len(model_oofs)):
            blend += weights[i] * model_oofs[i][cols]
        blend = np.clip(blend, 0, 1.0)
        blends[cols] = blend
        loss = oof_score(blends)
        return loss

In [ ]:
import optuna
pruner = optuna.pruners.MedianPruner(
        n_startup_trials=5,
        n_warmup_steps=0,
        interval_steps=1,
    )
sampler = optuna.samplers.TPESampler(seed=42)
study = optuna.create_study(direction="maximize",
                                pruner=pruner,
                                sampler=sampler,
                              #  study_name=study_name,
                              #  storage=f'sqlite:///{study_name}.db',
                                load_if_exists=True)

study.optimize(objective,
                   n_trials=2350,
                   timeout=None,
                   gc_after_trial=True,
                   n_jobs=-1)

trial = study.best_trial

In [ ]:
study.best_params

In [ ]:
study.best_trial
